In [0]:
print("Spark version", sc.version, spark.sparkContext.version, spark.version)
print("Python version", sc.pythonVer)

In [0]:
import requests

In [0]:
r = requests.get("https://timeseries.surge.sh/usd_to_eur.csv")

In [0]:
df = spark.read.csv(sc.parallelize(r.text.splitlines()), header=True, inferSchema=True)

In [0]:
display(df)

In [0]:
container = "raw"
storageAccount = "sourceblobstorageacc"
accessKey = "8nuZDSiapgVATVxV+fm3lwIFF0bhq45H490Dok/RrACg0MUo052M+NJ+o42ZKBslrEZPdFucjfOI+AStc9QTvQ=="

accountKey = "fs.azure.account.key.{}.blob.core.windows.net".format(storageAccount)

In [0]:
# Set the credentials to Spark configuration
spark.conf.set(
  accountKey,
  accessKey)

In [0]:
# Set the access key also in SparkContext to be able to access blob in RDD
# Hadoop configuration options set using spark.conf.set(...) are not accessible via SparkContext..
# This means that while they are visible to the DataFrame and Dataset API, they are not visible to the RDD API.

spark._jsc.hadoopConfiguration().set(
  accountKey,
  accessKey)

In [0]:
# Mount the drive for native python
inputSource = "wasbs://{}@{}.blob.core.windows.net".format(container, storageAccount)
mountPoint = "/mnt/" + container
extraConfig = {accountKey: accessKey}

print("Mounting: {}".format(mountPoint))

In [0]:
try:
  dbutils.fs.mount(
    source = inputSource,
    mount_point = str(mountPoint),
    extra_configs = extraConfig
  )
  print("=> Succeeded")
except Exception as e:
  if "Directory already mounted" in str(e):
    print("=> Directory {} already mounted".format(mountPoint))
  else:
    raise(e)

In [0]:
dbutils.fs.ls(".")

In [0]:
%fs

ls

In [0]:
inputFilePath = "wasbs://{}@{}.blob.core.windows.net/{}".format(container, storageAccount, "/usd_to_eur.csv")
df = spark.read.format("csv").load(inputFilePath, header=True, inferSchema=True)
display(df)

In [0]:
df.printSchema()

In [0]:
df.describe().show()

In [0]:
display(df)

Databricks visualization. Run in Databricks to view.

In [0]:
df.createOrReplaceTempView("currencyRate")

In [0]:
spark.sql("SELECT * FROM currencyRate").show()

In [0]:
spark.sql(" SELECT month(Date) as month, ROUND(AVG(Rate),2) as avg_rate \
            FROM currencyRate \
            GROUP BY month \
            ORDER BY month")\
    .show()

In [0]:
df2 = spark.sql(" SELECT year(Date) as year, ROUND(AVG(Rate),2) as avg_rate \
            FROM currencyRate \
            GROUP BY year \
            ORDER BY year")
display(df2)

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT year(Date) as year, ROUND(AVG(Rate),2) as avg_rate
            FROM currencyRate 
            GROUP BY year 
            ORDER BY year

In [0]:
import pyspark.sql.functions as f


DF = (df.groupBy(f.year("Date").alias("year"))
      .agg(f.count("Date").alias("count"), f.mean("rate").alias("mean"))
      .sort(f.desc("year")))
      
display(DF)
